In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms
import pdb

torch.set_printoptions(linewidth=120)
torch.set_grad_enabled(True) 

In [2]:
train_set = torchvision.datasets.FashionMNIST(
    root='./data'
    ,train=True
    ,download=True
    ,transform=transforms.Compose([
        transforms.ToTensor()
    ])
)

In [3]:
def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()


In [4]:
class Network(nn.Module):
    def __init__(self):
        super(Network,self).__init__()
        self.conv1 = nn.Conv2d(in_channels =1,out_channels =6,kernel_size =5)
        self.conv2 = nn.Conv2d(in_channels =6,out_channels =12,kernel_size =5)
        
        self.fc1 = nn.Linear(in_features = 12*4*4, out_features = 120)
        self.fc2 = nn.Linear(in_features = 120, out_features = 60)
        self.out = nn.Linear(in_features= 60, out_features=10)
        
    def forward(self, t):
    # (1) input layer
        t = t

    # (2) hidden conv layer
        t = self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

    # (3) hidden conv layer
        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

    # (4) hidden linear layer
        t = t.reshape(-1, 12 * 4 * 4)
        t = self.fc1(t)
        t = F.relu(t)

    # (5) hidden linear layer
        t = self.fc2(t)
        t = F.relu(t)

    # (6) output layer
        t = self.out(t)
    #t = F.softmax(t, dim=1)

        return t

In [5]:
network = Network()

train_loader = torch.utils.data.DataLoader(train_set, batch_size=100)
optimizer = optim.Adam(network.parameters(), lr=0.01)

In [6]:
for epoch in range(10):

    total_loss = 0
    total_correct = 0

    for batch in train_loader: # Get Batch
        images, labels = batch 

        preds = network(images) # Pass Batch
        loss = F.cross_entropy(preds, labels) # Calculate Loss

        optimizer.zero_grad()
        loss.backward() # Calculate Gradients
        optimizer.step() # Update Weights

        total_loss += loss.item()
        total_correct += get_num_correct(preds, labels)

    print(
        "epoch", epoch, 
        "total_correct:", total_correct, 
        "loss:", total_loss
    )

epoch 0 total_correct: 47224 loss: 338.7675912082195
epoch 1 total_correct: 51470 loss: 231.18222773075104
epoch 2 total_correct: 52012 loss: 212.5699834227562
epoch 3 total_correct: 52495 loss: 201.86561261117458
epoch 4 total_correct: 52686 loss: 196.37344471365213
epoch 5 total_correct: 52912 loss: 189.55152070522308
epoch 6 total_correct: 53042 loss: 187.19572585821152
epoch 7 total_correct: 53160 loss: 183.8989185988903
epoch 8 total_correct: 53223 loss: 181.42849112302065
epoch 9 total_correct: 53285 loss: 180.167987331748


In [7]:
@torch.no_grad()
def get_all_preds(model, loader):
    all_preds = torch.tensor([])
    for batch in prediction_loader:
        images, labels = batch

        preds = model(images)
        all_preds = torch.cat((all_preds, preds),dim=0)
    return all_preds

In [8]:
with torch.no_grad():
    prediction_loader = torch.utils.data.DataLoader(train_set, batch_size=100)
    train_preds = get_all_preds(network, prediction_loader)

In [9]:
preds_correct = get_num_correct(train_preds, train_set.targets)

print('total correct:', preds_correct)
print('accuracy:', preds_correct / len(train_set))


total correct: 53322
accuracy: 0.8887
